In [1]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\mohamed\\Desktop\\stage pfe Orange\\orangePlatform - Copie\\orangePlatform'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class Model_TrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    num_lstm_units: float
    learning_rate: float
    epochs: float
    early_stopping_patience: float
    nsteps: float

In [5]:
from orangePlatform.constants import *
from orangePlatform.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_best_model_trainer_config(self) -> Model_TrainerConfig:
        config = self.config.best_model_trainer
        params = self.params.ConvLSTM2D

        create_directories([config.root_dir])

        model_trainer_config = Model_TrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            num_lstm_units=params.num_lstm_units,
            learning_rate=params.learning_rate,
            epochs=params.epochs,
            early_stopping_patience=params.early_stopping_patience,
            nsteps=params.nsteps
                    
        )

        return model_trainer_config

In [7]:
import pandas as pd
import os
from orangePlatform import logger
import joblib

In [8]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, TimeDistributed
from keras.layers import ConvLSTM2D

In [9]:
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import yaml


In [10]:
def create_lstm_model_3_layers(num_lstm_units, learning_rate, nsteps):
    n_seq = 5
    n_steps = int(nsteps // n_seq)
    model = Sequential()
    model.add(ConvLSTM2D(filters=num_lstm_units, kernel_size=(1, 2), activation='relu', padding='same', input_shape=(n_seq, 1, n_steps, 1), return_sequences=True))
    model.add(ConvLSTM2D(filters=num_lstm_units//2, kernel_size=(1, 2), activation='relu', padding='same', return_sequences=True))
    model.add(ConvLSTM2D(filters=num_lstm_units//4, kernel_size=(1, 2), activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    return model

In [11]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [12]:
class BestModelTrainer:
    def __init__(self, config: Model_TrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train=train_data.values
        test=test_data.values

        hyperparams = {
            'num_lstm_units': [256],
            'learning_rate': [0.001],
            'epochs': [100],
            'early_stopping_patience': [10],
            'nsteps': [10]
        }
        best_param={'num_lstm_units': 256, 'learning_rate': 0.001, 'epochs': 180, 'early_stopping_patience': 10, 'nsteps': 15}
        loss=   1e+20
        # Hyperparameter search loop
        for num_lstm_units in hyperparams['num_lstm_units']:
            for learning_rate in hyperparams['learning_rate']:
                for epochs in hyperparams['epochs']:
                    for early_stopping_patience in hyperparams['early_stopping_patience']:
                        for nsteps in hyperparams['nsteps']:
                        

                            # Create and train the LSTM model
                            model = create_lstm_model_3_layers(num_lstm_units, learning_rate, nsteps)
                            X, y = split_sequence(train, nsteps)  # You need to define split_sequence
                            n_features = 1
                            n_seq =5
                            n_steps = int(nsteps // n_seq)
                            X = X.reshape((X.shape[0], n_seq, 1, n_steps, 1))
                            X_val, y_val = split_sequence(test, nsteps)
                            X_val = X_val.reshape((X_val.shape[0], n_seq, 1, n_steps, 1))
                            # Define early stopping callback
                            early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True)
                            # Train the model and log metrics using MLflow
                            history = model.fit(X, y, epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping])
                            if history.history['val_loss'][-1] < loss:
                                loss = history.history['val_loss'][-1]
                                best_param={'num_lstm_units': num_lstm_units, 'learning_rate': learning_rate, 'epochs': epochs, 'early_stopping_patience': early_stopping_patience, 'nsteps': nsteps}

        
           # Load the existing params.yaml file
        with open('params.yaml', 'r') as file:
            params = yaml.safe_load(file)

# Update the parameters in the params dictionary
        params['ConvLSTM2D'].update(best_param)

# Write the updated parameters back to the params.yaml file
        with open('params.yaml', 'w') as file:
            yaml.dump(params, file, default_flow_style=False)
                            




In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_best_model_trainer_config()
    model_trainer_config = BestModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-05 23:03:53,565: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-05 23:03:53,568: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-05 23:03:53,570: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-05 23:03:53,571: INFO: common: created directory at: artifacts]
[2024-06-05 23:03:53,573: INFO: common: created directory at: artifacts/best_model_trainer]
Epoch 1/100
10/10 [==============================] - 7s 112ms/step - loss: 0.2446 - val_loss: 0.2860
Epoch 2/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0858 - val_loss: 0.0308
Epoch 3/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0445 - val_loss: 0.0685
Epoch 4/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0367 - val_loss: 0.0295
Epoch 5/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0350 - val_loss: 0.0401
Epoch 6/100
10/10 [==============================] - 0s 48ms/step - loss: 0